In [1]:
import pandas as pd
movie = pd.read_csv("../db/movie_list.csv", index_col=0)
movie

,영화ID,영화명,개봉일,국적,장르,감독,배우
0,0,범죄도시2,2022-05-18,한국,"범죄,액션",이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형..."
1,1,아바타: 물의 길,2022-12-14,미국,"액션,어드벤처,SF,스릴러",제임스 카메론,"조 샐다나,샘 워싱턴,시고니 위버,스티븐 랭,케이트 윈슬렛"
2,2,탑건: 매버릭,2022-06-22,미국,액션,조셉 코신스키,"톰 크루즈,마일즈 텔러,제니퍼 코넬리,존 햄,에드 해리스,글렌 포웰,제이 엘리스,그..."
3,3,스파이더맨: 노 웨이 홈,2021-12-15,미국,"액션,어드벤처,SF",존 왓츠,"톰 홀랜드,젠데이아 콜먼,베네딕트 컴버배치,제이콥 배덜런,존 파브로,마리사 토메이"
4,4,한산: 용의 출현,2022-07-27,한국,"액션,드라마",김한민,"박해일,이지영,이정태,변요한,안성기,손현주,김성규,김성균,김향기,옥택연,공명,박지환..."
...,...,...,...,...,...,...,...
11278,11278,하나안,2012-10-11,한국,"드라마,범죄",박루슬란,"스타니슬라브 장,바호디르 무사에프,일벡 파이지에프,드미트리 엄,박루슬란"
11279,11279,허수아비들의 땅,2009-04-30,한국,드라마,노경태,"김선영,박원주,김화정,김두홍,신예슬,황상연,박현진,서민정,최경락,최광수,윤심경,임독..."
11280,11280,혹성탈출: 진화의 시작,2011-08-17,미국,"액션,SF,드라마",루퍼트 와이어트,"브라이언 콕스,제임스 프랭코,프리다 핀토"
11281,11281,후세: 말하지 못한 내 사랑,2013-03-28,일본,"애니메이션,드라마",미야지 마사유키,"코토부키 미나코,미야노 마모루,코니시 카츠유키"


In [2]:
movie_list = movie['영화명']
movie_list

0                  범죄도시2
1              아바타: 물의 길
2                탑건: 매버릭
3          스파이더맨: 노 웨이 홈
4              한산: 용의 출현
              ...       
11278                하나안
11279           허수아비들의 땅
11280       혹성탈출: 진화의 시작
11281    후세: 말하지 못한 내 사랑
11282     흐르는 강물을 어찌 막으랴
Name: 영화명, Length: 11283, dtype: object

In [3]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.edge.service import Service
from tqdm import tqdm
from urllib.parse import quote

In [4]:
url = "https://www.naver.com/"
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Edge('edgedriver', options=options)

driver.get(url)

C:\Users\gram15\AppData\Local\Temp\ipykernel_15376\3252289245.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('edgedriver', options=options)


In [4]:
movie = '범죄도시2'

In [6]:
driver.find_element(By.CSS_SELECTOR, '#query').send_keys([f'영화 {movie} 평점', Keys.ENTER])

In [7]:
# 리뷰/평점 리스트에서 관람객으로 설정되어있을 경우 전체 리뷰로 변경
soup = BeautifulSoup(driver.page_source, 'html.parser')

if  soup.select_one('._sorting_select_actual._trigger > a > span.menu._trigger_text').get_text() == '관람객':
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual._trigger').click()
    time.sleep(0.2)
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual > div > div > div > div > ul > li:nth-child(1) > a').click()
else:
    pass

if  soup.select_one('._sorting_select_order > a > span.menu._trigger_text').get_text() == '최신순':
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_order._trigger').click()
    time.sleep(0.2)
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_order > div > div > div > div > ul > li:nth-child(1) > a').click()
else:
    pass

# 스포일러 허용 버튼
driver.find_element(By.CSS_SELECTOR, '._spoiler_switch > button.btn_area_auto').click()

In [8]:
scroll = driver.find_element(By.CSS_SELECTOR, '._cm_content_area_rating > div > div:nth-child(2) > div.lego_review_list._scroller')

while True :
	# 스크롤 전의 마지막 리뷰 내용
	prev_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
	# 첫번째로 스크롤 내리기
	driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", scroll)
	# 시간대기				
	time.sleep(0.5)
	# 스크롤 후 마지막 리뷰 내용
	soup = BeautifulSoup(driver.page_source, 'html.parser')
	current_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
	time.sleep(0.5)
	# 스크롤 전과 후의 마지막 리뷰가 같으면 종료
	if current_review == prev_review:
		False
		break

In [9]:
# 리뷰만 추출
soup = BeautifulSoup(driver.page_source, 'html.parser')
review_list = soup.select("div.lego_review_list._scroller > ul > li > div.area_review_content > div > span.desc._text")
print(len(review_list))

50


In [10]:
review_list[0:5]

[<span class="desc _text">어우 의자에 앉아서 스크린으로 보는데도 느껴지는 타격감 </span>,
 <span class="desc _text">범죄도시3이 기대돼요! </span>,
 <span class="desc _text">범죄도시3가 기대됩니다 </span>,
 <span class="desc _text">마동석외에도 연기 좋아요 </span>,
 <span class="desc _text">너무너무 잔인했음... </span>]

In [11]:
for org in review_list:
    review_before = str(org)
    fwd = review_before.split('">')[1]
    bck = fwd.split(' </')[0]
    print(bck)

어우 의자에 앉아서 스크린으로 보는데도 느껴지는 타격감
범죄도시3이 기대돼요!
범죄도시3가 기대됩니다
마동석외에도 연기 좋아요
너무너무 잔인했음...
아무 생각없이 볼 수 있는 영화라 좋았음
범죄영화에서 오락영화로 장르변경이 되어버린 느낌
영화값이 아깝지 않은 영화
범죄액션영화 깔끔하게 재미있다
천만의 품격! 이 정도는 되야 천만을 갈 수 있는 영화! 너무 재밌었다
어우 의자에 앉아서 스크린으로 보는데도 느껴지는 타격감
범죄도시3이 기대돼요!
범죄도시3가 기대됩니다
마동석외에도 연기 좋아요
너무너무 잔인했음...
아무 생각없이 볼 수 있는 영화라 좋았음
범죄영화에서 오락영화로 장르변경이 되어버린 느낌
영화값이 아깝지 않은 영화
범죄액션영화 깔끔하게 재미있다
천만의 품격! 이 정도는 되야 천만을 갈 수 있는 영화! 너무 재밌었다
마동석 그는 진짜 최고!! 타격음이 장난아님ㅋㅋㅋ 범죄도시 10편까지 가즈아!
훌륭한 오락영화...
개인적으로 빌런은 1편보다 약했지만 1편보다 웃겼어요
손석구 연기 좋습니다 3편 기대됩니다
재미와 액션이 적절하게 가미된 영화. 속편이 원작 못 따라간다는 편견을 시원하게 깨버린 영화인데 범죄자들 머리도 시원하게 깸.
이정도면 훌륭하지 시원한 액션~
마동석 액션 타격감은 매우 좋으나 그렇게 재밌나...?싶기는 함
1보다는... 생각보다 그냥 그래요
사이다 영화. 진짜 7편까지 나오려나
3편 이제 곧 나온다 기대중..
3편이 빨리 나왓우면 합니다 ㅎㅎ
영화적 재미에 충실한 작품
이 시리즈화 완전 찬성이요
배우들의 연기가 좋았고 액션이 시원시원해서 좋았어요
손석구의 새로운 모습 좋았다
쓰리가 얼른 나오면 좋겠네요
너무 재밌다 즐기면서 볼 수 있는 영화
장르에 충실하다 재미도 있다
한국 액션이 마동석으로 인해서 단단해진 느낌!
속시원했음ㅎㅎ 더 때려주길

선과 악의 맞서는 모습이 긴장감있으면서도 중간중안 코믹하게 그려져 재미있게 관람함.
팝콘 무비 정석인 듯
the roundup , 2022
유쾌통쾌한 영화였어요. 정말 재밌게 잘 봤습니다.
짱

In [12]:
# 평점 요소
soup = BeautifulSoup(driver.page_source, 'html.parser')
rating_before = soup.select('div.area_title_box > div > div.area_text_box')
print(len(rating_before))

50


In [13]:
for org_rv in rating_before:
    rating_org = str(org_rv)
    fwd_del_rv = rating_org.split('중)</span>')[1]
    bck_del_rv = fwd_del_rv.split('</div')[0]
    print(bck_del_rv)

7
8
5
8
8
6
8
8
9
10
7
8
5
8
8
6
8
8
9
10
8
10
9
10
9
8
7
2
8
10
10
7
8
9
6
10
7
7
7
8
7
9
7
9
10
8
6
10
10
10


In [5]:
url = "https://www.naver.com/"
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Edge('edgedriver', options=options)


driver.get(url)
time.sleep(2)

rating_before = [] # 평점 임시 리스트
review_before = [] # 리뷰 임시 리스트
success_cnt = 0
fail_cnt = 0
total_cnt = 0

rating_list = []
review_list = []
movie_title = []
fail_list = [] # 크롤링 실패 리스트

for movie in movie_list[:501]:
    try:
        try:
            driver.find_element(By.CSS_SELECTOR, '#query').send_keys([f'영화 {movie} 평점', Keys.ENTER])
            time.sleep(1)
        except:
            driver.find_element(By.CSS_SELECTOR, '#nx_query').send_keys([f'영화 {movie} 평점', Keys.ENTER])
            time.sleep(1)

        # 리뷰/평점 리스트에서 관람객으로 설정되어있을 경우 전체 리뷰로 변경
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        time.sleep(1)
        if  soup.select_one('._sorting_select_actual._trigger > a > span.menu._trigger_text').get_text() == '관람객':
            driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual._trigger').click()
            time.sleep(0.5)
            driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual > div > div > div > div > ul > li:nth-child(1) > a').click()
        else:
            pass

        if  soup.select_one('._sorting_select_order > a > span.menu._trigger_text').get_text() == '최신순':
            driver.find_element(By.CSS_SELECTOR, '._sorting_select_order._trigger').click()
            time.sleep(0.5)
            driver.find_element(By.CSS_SELECTOR, '._sorting_select_order > div > div > div > div > ul > li:nth-child(1) > a').click()
        else:
            pass
    
        # 스포일러 허용 버튼
        driver.find_element(By.CSS_SELECTOR, '._spoiler_switch > button.btn_area_auto').click()

        # 셀레니움 스크롤 끝까지 내려도 계속 내리는 페이지라면
        scroll = driver.find_element(By.CSS_SELECTOR, '._cm_content_area_rating > div > div:nth-child(2) > div.lego_review_list._scroller')
        while True :
            # 스크롤 전의 마지막 리뷰 내용
            prev_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
            # 첫번째로 스크롤 내리기
            driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", scroll)
            # 시간대기				
            time.sleep(2)
            # 스크롤 후 마지막 리뷰 내용
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            current_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
            time.sleep(1)
            # 스크롤 전과 후의 마지막 리뷰가 같으면 종료
            if current_review == prev_review:
                False
                break
        time.sleep(1)
        print("=" * 50)
        print("영화제목 : ", movie)
            
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 평점 요소
        rating_before = soup.select('div.area_title_box > div > div.area_text_box')
        time.sleep(0.5)
        for org_rv in rating_before:
            rating_org = str(org_rv)
            fwd_del_rv = rating_org.split('중)</span>')[1]
            bck_del_rv = fwd_del_rv.split('</div')[0]
            rating_list.append(int(bck_del_rv))
        print("평점개수 : ", len(rating_before))

        # 리뷰만 추출
        review_before = soup.select("div.lego_review_list._scroller > ul > li > div.area_review_content > div > span.desc._text")
        time.sleep(0.5)
        for org in review_before:
            review_org = str(org)
            fwd_del = review_org.split('">')[1]
            bck_del = fwd_del.split(' </')[0]
            review_list.append(bck_del)
            movie_title.append(movie) # 리뷰 구분을 위한 영화 타이틀 추가
        print("리뷰개수 : ", len(review_before))
        success_cnt +=1
        print(f"성공 누적 건수 : {success_cnt}건")
        print("=" * 50)
        review_before = [] # 리뷰 임시 리스트 초기화
        url = driver.current_url
        driver.find_element(By.CSS_SELECTOR, '#nx_query').clear()
    
    except AttributeError as error:
        fail_cnt += 1
        print("=" * 50)
        print("영화제목 : ", movie)
        fail_list.append(movie)
        print("평점 항목 없음")
        print(error)
        print(f"실패 누적 건수 : {fail_cnt}건")
        print("=" * 50)
        url = driver.current_url
        driver.find_element(By.CSS_SELECTOR, '#nx_query').clear()
        pass

    finally:
        total_cnt += 1
        print("=" * 50)
        print(f"총 누적 건수 : {total_cnt}건")
        print("=" * 50)


C:\Users\gram15\AppData\Local\Temp\ipykernel_8560\1056336168.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('edgedriver', options=options)


영화제목 :  범죄도시2
평점개수 :  90
리뷰개수 :  90
성공 누적 건수 : 1건
총 누적 건수 : 1건
영화제목 :  아바타: 물의 길
평점개수 :  30
리뷰개수 :  30
성공 누적 건수 : 2건
총 누적 건수 : 2건
영화제목 :  탑건: 매버릭
평점개수 :  30
리뷰개수 :  30
성공 누적 건수 : 3건
총 누적 건수 : 3건
영화제목 :  스파이더맨: 노 웨이 홈
평점개수 :  50
리뷰개수 :  50
성공 누적 건수 : 4건
총 누적 건수 : 4건
영화제목 :  한산: 용의 출현
평점개수 :  20
리뷰개수 :  20
성공 누적 건수 : 5건
총 누적 건수 : 5건
영화제목 :  공조2: 인터내셔날
평점개수 :  90
리뷰개수 :  90
성공 누적 건수 : 6건
총 누적 건수 : 6건
영화제목 :  닥터 스트레인지: 대혼돈의 멀티버스
평점개수 :  20
리뷰개수 :  20
성공 누적 건수 : 7건
총 누적 건수 : 7건
영화제목 :  헌트
평점개수 :  20
리뷰개수 :  20
성공 누적 건수 : 8건
총 누적 건수 : 8건
영화제목 :  모가디슈
평점개수 :  20
리뷰개수 :  20
성공 누적 건수 : 9건
총 누적 건수 : 9건
영화제목 :  올빼미
평점개수 :  30
리뷰개수 :  30
성공 누적 건수 : 10건
총 누적 건수 : 10건
영화제목 :  영웅
평점개수 :  80
리뷰개수 :  80
성공 누적 건수 : 11건
총 누적 건수 : 11건
영화제목 :  이터널스
평점개수 :  20
리뷰개수 :  20
성공 누적 건수 : 12건
총 누적 건수 : 12건
영화제목 :  블랙 위도우
평점개수 :  40
리뷰개수 :  40
성공 누적 건수 : 13건
총 누적 건수 : 13건
영화제목 :  가디언즈 오브 갤럭시: Volume 3
평점개수 :  20
리뷰개수 :  20
성공 누적 건수 : 14건
총 누적 건수 : 14건
영화제목 :  쥬라기 월드: 도미니언
평점개수 :  20
리뷰개수 :  20
성공 누적 건수 : 15건
총 누적 건

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#nx_query"}
  (Session info: MicrosoftEdge=113.0.1774.50)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF626674AF2+67490]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF62660B262+782450]
	(No symbol) [0x00007FF6263AC646]
	(No symbol) [0x00007FF6263EF972]
	(No symbol) [0x00007FF6263EFB8A]
	(No symbol) [0x00007FF626429817]
	(No symbol) [0x00007FF62640DF8F]
	(No symbol) [0x00007FF6263E3751]
	(No symbol) [0x00007FF626426BC5]
	(No symbol) [0x00007FF62640DD23]
	(No symbol) [0x00007FF6263E2794]
	(No symbol) [0x00007FF6263E19B0]
	(No symbol) [0x00007FF6263E2F04]
	Microsoft::Applications::Events::ILogManager::DispatchEventBroadcast [0x00007FF6268390F3+1301571]
	(No symbol) [0x00007FF62646B951]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x00007FF6265546B1+33985]
	Microsoft::Applications::Events::EventProperty::~EventProperty [0x000

In [6]:
result = pd.DataFrame([rating_list, review_list, movie_title])

In [7]:
result = result.transpose()
result

,0,1,2
0,10,이 영화의 속편은 100% 성공이다. 원래 한국영화들 속편은 잘 안되고 속편 나오면...,범죄도시2
1,10,시리즈로 계속 나왔으면 좋겠다. 마동석한테 최적화된 작품이다.,범죄도시2
2,8,니가 강해상이냐? 아뇨. 구씬데요?,범죄도시2
3,10,전편을 보고 가야 장이수 얼굴만 봐도 웃음이 나오는 이유를 알 수 있습니다:),범죄도시2
4,9,와…. 손석구 연기 진짜 미쳣다…오바하는 범죄자연기가 아니라 ㄹㅇ 범죄자같음,범죄도시2
...,...,...,...
9442,4,더 재밌게 만들 수 있을 소재인데 너무 지루함,민스미트 작전
9443,6,예상외의 영화였지만 나름 좋았습니다,민스미트 작전
9444,8,잔잔해서 임팩트는 없는듯 미쟝센은 훌륭,민스미트 작전
9445,7,후반으로갈수록 힘이빠진다,민스미트 작전
